# Otic Mesenchyme and Sensory Epithelium Cell-Cell Communication Analysis

Prepared by: Abel P. David (Northwestern University)

Date: 11/6/2025

Prepared for: Ippei Kishimoto and Alan Cheng (Stanford University)

In [ ]:
library(CellChat)
library(patchwork)
library(NMF)
library(circlize)
library(ComplexHeatmap)
library(Seurat)
library(Matrix)
library(magrittr)
library(tidyverse)
library(SingleCellExperiment)
library(remotes)
library(renv)
library(future)
library(furrr)
library(SeuratDisk)
library(devtools)
library(Seurat)
library(SeuratDisk)
library(harmony)

# E13/E14 Combination

In [ ]:
data_name = "E13E14_PSC_Separated"
so_e13 <- readRDS("sun_2022_e13.rds")
so_e14 <- readRDS("kolla_2020_e14.rds")

In [ ]:
DimPlot(so_e13, group.by = "cluster")
so_e13$cluster_combined <- so_e13$cluster

In [ ]:
so_e13 <- subset(so_e13, subset = cluster %in% c("mesenchymal_cells-C1"))

In [ ]:
DimPlot(so_e13, group.by = "cluster")

In [ ]:
DimPlot(so_e14, group.by = "cell_type")
so_e14$cluster_combined <- so_e14$cell_type

In [ ]:
so_e13$batch <- "E13"
so_e14$batch <- "E14"

In [ ]:
so <- merge(so_e13, y = so_e14, merge.data = TRUE)
so[["RNA"]] <- JoinLayers(so[["RNA"]])

In [ ]:
so <- NormalizeData(so)
so <- FindVariableFeatures(so, nfeatures = 3000)
so <- ScaleData(so, verbose = FALSE)
so <- RunPCA(so, npcs = 30, verbose = FALSE)

In [ ]:
so <- RunHarmony(object = so, group.by.vars = "batch", reduction.use = "pca", plot_convergence = TRUE)

In [ ]:
so <- RunUMAP(so, reduction = "harmony", dims = 1:30)
so <- FindNeighbors(so, reduction = "harmony", dims = 1:30)
so <- FindClusters(so, resolution = 1.0)

In [ ]:
DimPlot(so, group.by = "cluster_combined", label = TRUE)

In [ ]:
so@meta.data <- so@meta.data %>%
  mutate(
    final_cluster = case_when(
      cluster_combined %in% c("OHC", "IHC") ~ "HC",
      cluster_combined %in% c("LPro", "MPro") ~ "PsC",
      cluster_combined %in% c("mesenchymal_cells-C1") ~ "POM",
      cluster_combined %in% c("GER", "GER/Hmgn2", "IdC", "LER/Bmp4_1", "LER/Bmp4_2", "LER/Fst", "Oc90_3", "Oc90_4", "Oc90/Otoa", "Oc90/Sparcl1") ~ "Epithelial",
      TRUE ~ "None"
    )
)

In [ ]:
so@meta.data <- so@meta.data %>%
  mutate(
    sub_cluster = case_when(
      cluster_combined %in% c("OHC", "IHC") ~ "HC",
      cluster_combined %in% c("LPro", "MPro") ~ "PsC",
      cluster_combined %in% c("mesenchymal_cells-C1") ~ "POM",
      cluster_combined %in% c("GER", "GER/Hmgn2", "IdC") ~ "GER",
      cluster_combined %in% c("LER/Bmp4_1", "LER/Bmp4_2", "LER/Fst") ~ "LER",
      cluster_combined %in% c("Oc90_3", "Oc90_4", "Oc90/Otoa", "Oc90/Sparcl1") ~ "Roof",
      TRUE ~ "None"
    )
)

In [ ]:
DimPlot(so, group.by = "sub_cluster")

In [ ]:
pdf(paste0(data_name, "_Sub_Cluster_UMAP.pdf"), width = 6, height = 5, family = "Helvetica")
DimPlot(so, group.by = "sub_cluster")
dev.off()

In [ ]:
DimPlot(so, group.by = "final_cluster", label = TRUE)

In [ ]:
saveRDS(so, file = "e13_e14_harmony_separated.rds")